<a href="https://colab.research.google.com/github/Rango-Zhang-Hang/NTU-Visual-Recognition-via-Vision-Language-Model-Transfer/blob/main/MasterProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#load google drive
from google.colab import drive
drive.mount('/content/drive')

import os
#%cd drive/MyDrive
os.chdir('/content/drive/MyDrive')
!ls
#load under ~/MyDrive

#delete folders DO NOT DELETE through google drive, it will only delete the directory, but leave orphan files!
#os.chdir('/content/drive/MyDrive/data/lyft/')
#!ls
#%rm -rf train_maps
#!mv "v1.01-train/v1.01-train.zip" "v1.01-train.zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
config	  MasterProject				    Untitled0.ipynb
data	  Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
Fast-BEV  mmdetection3d


In [33]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
!ls
!chmod +x Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
%env PYTHONPATH=
!./Miniconda3-py38_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local

!which conda
!conda --version
!which python
!python -V
import sys
_ = (sys.path.append("/usr/local/lib/python3.8/site-packages"))
sys.path
!conda env list

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# creat conda env
!conda create --name hh python=3.8 -y
!source activate hh

#install packages
!source activate hh && pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!source activate hh && pip install -U openmim
!source activate hh && mim install mmengine
!source activate hh && mim install 'mmcv>=2.0.0rc4'
!source activate hh && mim install 'mmdet>=3.0.0'

!conda env list

#install mmdet3d
os.chdir('/content/drive/MyDrive')
!ls
!git clone https://github.com/open-mmlab/mmdetection3d.git -b dev-1.x
# "-b dev-1.x" means checkout to the `dev-1.x` branch.
%cd mmdetection3d
!source activate hh && pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in edtiable mode,
# thus any local modifications made to the code will take effect without reinstallation.

#install other packages
os.chdir('/content/drive/MyDrive/Fast-BEV')
!ls
!source activate hh && conda install -c conda-forge pretty_errors -y
!source activate hh && pip install -r requirements.txt

--2023-07-27 05:27:14--  https://repo.anaconda.com/miniconda/Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

config	  MasterProject				    Untitled0.ipynb
data	  Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
Fast-BEV  mmdetection3d
env: PYTHONPATH=
PREFIX=/usr/local
Unpacking payload ...
                                                                               
Installing base environment...





Preparing transaction: - done
Executing transaction: | done
installation finished.
/usr/local/bin/conda
conda 23.5.2
/usr/local/bin/python
Python 3.8.17
# conda environments:
#
base                     /usr/local
hh                       /usr/local/envs/hh

Solving environment: /

The structure should be like:
```
|MyDrive
|-config
|-data
  |-lyft
|Fast-BEV
  |-mmdet3d-dlc
```

Besides, remember to actiavte conda_env 'hh' for every block using it.

In [54]:
#load google drive
from google.colab import drive
drive.mount('/content/drive')
#activate conda env
!source activate hh && conda env list

#prepare the annotation files for lyft: original->fastbev type
'''
#original lyft
os.chdir('/content/drive/MyDrive/mmdetection3d')
!ls
!python tools/create_data.py lyft --root-path ../data/lyft --out-dir ../data/lyft --extra-tag lyft --version v1.01
!python tools/dataset_converters/lyft_data_fixer.py --version v1.01 --root-folder ../data/lyft
'''
'''
#fastbev lyft
import sys
sys.path.append('/content/drive/MyDrive/Fast-BEV')
print(sys.path)
os.chdir('/content/drive/MyDrive/Fast-BEV')
!ls
!source activate hh && python tools/custom_create_data.py lyft
'''

#test
import sys
sys.path.append('/content/drive/MyDrive/Fast-BEV')
os.chdir('/content/drive/MyDrive/Fast-BEV')
print("loading testing...\n")
!source activate hh && python -u tools/'custom_test.py' configs/fastbev.py weights/standard.pth --launcher="none"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# conda environments:
#
base                     /usr/local
hh                    *  /usr/local/envs/hh

loading testing...

['/content/drive/MyDrive/Fast-BEV/tools', '/usr/local/envs/hh/lib/python38.zip', '/usr/local/envs/hh/lib/python3.8', '/usr/local/envs/hh/lib/python3.8/lib-dynload', '/usr/local/envs/hh/lib/python3.8/site-packages', '/content/drive/MyDrive/mmdetection3d', '/tmp/tmpkx7npo_u', '/content/drive/MyDrive/Fast-BEV']
False
07/27 07:44:01 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.17 (default, Jul  5 2023, 21:04:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1599984244
    GPU 0: Tesla V100-SXM2-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1)